# Develop a Custom Object Detector - Mask-RCNN Version

###### Let's Upgrade Anaconda if you have it installed

In [1]:
# Enable to update/upgrade anaconda
#!conda update conda
#!conda upgrade anaconda

###### Now install the required packaget to continue the flow

In [2]:
# Enable if you did not installed the packages
#!pip install -r requirements.txt

###### Next we will be doing some neccesary imports

In [42]:
import os
import fnmatch
import pandas as pd
import numpy as np

from xml.etree import ElementTree as et
from urllib.request import urlretrieve

In [4]:
xml_train_path  = os.path.join('Dataset','Train')
xml_train_files = fnmatch.filter(os.listdir(xml_train_path), "*.xml")
xml_train_files_full_path = [os.path.join(xml_train_path, p) for p in xml_train_files]

xml_test_path   = os.path.join('Dataset', 'Test')
xml_test_files  = fnmatch.filter(os.listdir(xml_test_path), "*.xml")
xml_test_files_full_path = [os.path.join(xml_test_path, p) for p in xml_test_files]

###### see a dataset xml file path

In [5]:
xml_test_files_full_path[:5]

['Dataset\\Test\\0710_nc_red_fox_jpg.xml',
 'Dataset\\Test\\0_1024px-Black_fox.xml',
 'Dataset\\Test\\0_badger1_result.xml',
 'Dataset\\Test\\0_Eurasian-badger-Meles-meles-emerging-from-sett-England.xml',
 'Dataset\\Test\\1 (1).xml']

In [6]:
xml_train_files_full_path[:5]

['Dataset\\Train\\asian-badger.xml',
 'Dataset\\Train\\baby-badger-drinking-figurine.xml',
 'Dataset\\Train\\baby-fox.xml',
 'Dataset\\Train\\badger (1).xml',
 'Dataset\\Train\\badger (10).xml']

###### see how many images are in the train and test

In [7]:
ntrain = len(xml_train_files)
ntest  = len(xml_test_files)
print(f'There are {ntrain} images for training.\nThera are {ntest} images for validation.')

There are 236 images for training.
Thera are 60 images for validation.


###### test xml extraction for train and test data

In [8]:
tree = et.parse(xml_train_files_full_path[0])
root = tree.getroot()

In [9]:
print(et.tostring(root, encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<annotation>
	<folder>badger_resized</folder>
	<filename>asian-badger.jpg</filename>
	<path>Dataset\Train\asian-badger.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>720</width>
		<height>514</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>badger</name>
		<pose>Unspecified</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>14</xmin>
			<ymin>1</ymin>
			<xmax>711</xmax>
			<ymax>450</ymax>
		</bndbox>
	</object>
</annotation>


In [10]:
for fname in root.iter('filename'):
    print(fname.text)

asian-badger.jpg


In [11]:
for path in root.iter('path'):
    print(path.text)

Dataset\Train\asian-badger.jpg


In [12]:
for s in root.iter('size'):
    w = int(s.find('width').text)
    h = int(s.find('height').text)
    print(w, h)

720 514


In [13]:
for box in root.findall('.//bndbox'):
    xmin = int(box.find('xmin').text)
    ymin = int(box.find('ymin').text)
    xmax = int(box.find('xmax').text)
    ymax = int(box.find('ymax').text)
    print(xmin, ymin, xmax, ymax)

14 1 711 450


In [14]:
for class_ in root.findall('.//object'):
    name = class_.find('name').text
    print(name)

badger


###### treat xml data

In [15]:
# Accomodate the path
for xmlpath in xml_train_files_full_path:
    tree = et.parse(xmlpath)
    root = tree.getroot()
    for fname in root.iter('filename'):
        name = fname.text
    for path in root.iter('path'):
        path.text = os.path.join(xml_train_path, name)
    tree.write(xmlpath)

In [16]:
# Accomodate the path
for xmlpath in xml_test_files_full_path:
    tree = et.parse(xmlpath)
    root = tree.getroot()
    for fname in root.iter('filename'):
        name = fname.text
    for path in root.iter('path'):
        path.text = os.path.join(xml_test_path, name)
    tree.write(xmlpath)

In [17]:
for path in root.iter('path'):
    print(path.text)

Dataset\Test\_fox_16x9.jpg


###### Make an array to feed later a dataframe

In [18]:
values_train = []
for xmlpath in xml_train_files_full_path:
    tree = et.parse(xmlpath)
    root = tree.getroot()
    for path in root.iter('path'):
        imagepath = path.text
    for b in root.findall('.//bndbox'):
        xmin = int(b.find('xmin').text)
        ymin = int(b.find('ymin').text)
        xmax = int(b.find('xmax').text)
        ymax = int(b.find('ymax').text)
    for class_ in root.findall('.//object'):
        class_name = class_.find('name').text
    values_train.append([imagepath, xmin, ymin, xmax, ymax, class_name])

In [19]:
values_train[0], values_train[1], values_train[2], values_train[3], values_train[4]

(['Dataset\\Train\\asian-badger.jpg', 14, 1, 711, 450, 'badger'],
 ['Dataset\\Train\\baby-badger-drinking-figurine.jpg',
  16,
  72,
  720,
  436,
  'badger'],
 ['Dataset\\Train\\baby-fox.jpg', 187, 76, 642, 425, 'fox'],
 ['Dataset\\Train\\badger (1).jpg', 7, 30, 628, 485, 'badger'],
 ['Dataset\\Train\\badger (10).jpg', 63, 22, 720, 478, 'badger'])

In [20]:
values_test = []
for xmlpath in xml_test_files_full_path:
    tree = et.parse(xmlpath)
    root = tree.getroot()
    for path in root.iter('path'):
        imagepath = path.text
    for b in root.findall('.//bndbox'):
        xmin = int(b.find('xmin').text)
        ymin = int(b.find('ymin').text)
        xmax = int(b.find('xmax').text)
        ymax = int(b.find('ymax').text)
    for class_ in root.findall('.//object'):
        class_name = class_.find('name').text
    values_test.append([imagepath, xmin, ymin, xmax, ymax, class_name])

In [21]:
values_test[0], values_test[1], values_test[2], values_test[3], values_test[4]

(['Dataset\\Test\\0710_nc_red_fox_jpg.jpg', 9, 59, 646, 374, 'fox'],
 ['Dataset\\Test\\0_1024px-Black_fox.jpg', 31, 52, 666, 423, 'fox'],
 ['Dataset\\Test\\0_badger1_result.jpg', 42, 100, 582, 456, 'badger'],
 ['Dataset\\Test\\0_Eurasian-badger-Meles-meles-emerging-from-sett-England.jpg',
  274,
  119,
  602,
  508,
  'badger'],
 ['Dataset\\Test\\1 (1).jpg', 236, 6, 658, 261, 'badger'])

###### make dataframes and write to csv

In [22]:
train_df = pd.DataFrame(values_train)
train_df.head()

,0,1,2,3,4,5
0,Dataset\Train\asian-badger.jpg,14,1,711,450,badger
1,Dataset\Train\baby-badger-drinking-figurine.jpg,16,72,720,436,badger
2,Dataset\Train\baby-fox.jpg,187,76,642,425,fox
3,Dataset\Train\badger (1).jpg,7,30,628,485,badger
4,Dataset\Train\badger (10).jpg,63,22,720,478,badger


In [23]:
test_df = pd.DataFrame(values_test)
test_df.head()

,0,1,2,3,4,5
0,Dataset\Test\0710_nc_red_fox_jpg.jpg,9,59,646,374,fox
1,Dataset\Test\0_1024px-Black_fox.jpg,31,52,666,423,fox
2,Dataset\Test\0_badger1_result.jpg,42,100,582,456,badger
3,Dataset\Test\0_Eurasian-badger-Meles-meles-eme...,274,119,602,508,badger
4,Dataset\Test\1 (1).jpg,236,6,658,261,badger


In [24]:
column_names = ['image_path','xmin','ymin', 'xmax','ymax','class_name']
train_df.columns = column_names
test_df.columns = column_names
test_df.head()

,image_path,xmin,ymin,xmax,ymax,class_name
0,Dataset\Test\0710_nc_red_fox_jpg.jpg,9,59,646,374,fox
1,Dataset\Test\0_1024px-Black_fox.jpg,31,52,666,423,fox
2,Dataset\Test\0_badger1_result.jpg,42,100,582,456,badger
3,Dataset\Test\0_Eurasian-badger-Meles-meles-eme...,274,119,602,508,badger
4,Dataset\Test\1 (1).jpg,236,6,658,261,badger


In [26]:
TRAIN_ANNOTATIONS = 'train_annotations.csv'
TEST_ANNOTATIONS  = 'test_annotations.csv' 
CLASSES           = 'classes.csv'

class_names = ['fox', 'badger']

with open(CLASSES, 'w') as classes:
    for i, cl in enumerate(class_names):
        classes.write(f'{cl}, {i}\n')


train_df.to_csv(TRAIN_ANNOTATIONS, index=False, header=False)
test_df.to_csv(TEST_ANNOTATIONS, index=False, header=False)

### Download the Repo and Weights

In [27]:
!git clone https://github.com/fizyr/keras-retinanet

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [28]:
BASE_MODEL_PATH = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
BASE_MODEL_NAME = 'resnet50_coco_best_v2.1.0.h5'

urlretrieve(BASE_MODEL_PATH, BASE_MODEL_NAME)

('resnet50_coco_best_v2.1.0.h5', <http.client.HTTPMessage at 0x3381f13508>)

In [ ]:
%cd keras-retinanet
!python setup.py install
%cd ..

In [1]:
!mkdir snapshot

In [47]:
BATCH_SIZE = 4                           # 4 images per batch
STEPS      = str(int(ntrain/BATCH_SIZE)) # 236/4 = 59 steps, str to pass an ascii to command line
EPOCHS     = str(10)                     # 10 rounds of the whole image dataset, str to pass an ascii to command line
WEIGHTS    = BASE_MODEL_NAME             # same as <current_working_directory>/resnet50_coco_best_v2.1.0.h5
SNAPSHOT   = 'snapshot'                  # same as <current_working_directory>/snapshot
TRAIN_BASE = TRAIN_ANNOTATIONS           # same as <current_working_directory>/train_annotations.csv
TEST_BASE  = TEST_ANNOTATIONS            # same as <current_working_directory>/test_annotations.csv
#CLASSES =                              # same as <current_working_directory>/classes.csv
BATCH_SIZE = str(BATCH_SIZE)             # normally is a number, but in command line is an ascii

In [ ]:
!retinanet-train \
--batch-size $BATCH_SIZE \
--steps $STEPS --epochs $EPOCHS \
--weights $WEIGHTS \
--snapshot-path $SNAPSHOT \
csv $TRAIN_BASE $CLASSES > train_log.txt   # see the log at <current_working_directory>/log.txt


KeyboardInterrupt


KeyboardInterrupt



In [ ]:
NAME             = 'resnet50' + '_csv_' + EPOCHS + '.h5' # the last model name at <current_working_directory>/resnet50_csv_10.h5
INPUT_MODEL_NAME =  os.path.join(SNAPSHOT, NAME)  # same as <current_working_directory>/snapshot/resnet10_csv_10.h5
OUTPUT_MODEL_NAME= 'output.h5'                    # the output name <current_orking_directory>

!retinanet-convert-model $INPUT_MODEL_PATH $OUTPUT_MODEL_NAME